In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
# config
TRAIN_DATA_PATH = "data/train.csv"

# load data
df_train = pd.read_csv(TRAIN_DATA_PATH)
print("original shape : " + str(df_train.shape))

# remove id column
df_train.drop('Id', axis=1, inplace=True)
print("shape after dropping id : " + str(df_train.shape))

In [ ]:
# analyze target variable
sns.distplot(df_train['SalePrice'])
plt.show()

print(df_train['SalePrice'].describe(), '\n')
print("Skewness: %f" % df_train['SalePrice'].skew())
print("Kurtosis: %f" % df_train['SalePrice'].kurt())

In [ ]:
# 可选，看模型需要
# log transformation on target variable
# df_train['SalePrice'] = np.log1p(df_train['SalePrice'])
# sns.distplot(df_train['SalePrice'])
# plt.show()

In [ ]:
# check outlier
plt.scatter(df_train.GrLivArea, df_train.SalePrice, c = "blue", marker = "s")
plt.title("Outliers")
plt.xlabel("GrLivArea")
plt.ylabel("SalePrice")
plt.show()

# remove GrLivArea >= 4000 following https://ww2.amstat.org/publications/jse/v19n3/decock.pdf
df_train = df_train[df_train.GrLivArea < 4000]

In [ ]:
# check missing data
missing_num = df_train.isnull().sum().sort_values(ascending=False)
missing_percentage = (df_train.isnull().sum()/df_train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([missing_num, missing_percentage], axis=1, keys=['Missing num', 'Percent'])
missing_data.head(20)

In [ ]:
print("shape before handling missing data : ", df_train.shape)

# handle missing data
# drop columns with too many missing values
df_train = df_train.drop((missing_data[missing_data['Percent'] > 0.15]).index, 1)
print("shape after dropping columns : ", df_train.shape)

# drop rows with missing values because the small amount
df_train = df_train.dropna()
print("shape after dropping rows : ", df_train.shape)

print('remaining null : ', df_train.isnull().sum().max())

In [ ]:
# correlation analysis
corrmat = df_train.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, vmax=.8, square=True);
plt.show()

# look closer to top k correlated variables
k = 10
cols = corrmat.nlargest(k, 'SalePrice')['SalePrice'].index
cm = np.corrcoef(df_train[cols].values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

In [ ]:
# 纯可视化用，很慢所以先注释掉
# scatterplot to visualize the correlation between variables
# sns.set()
# cols = ['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt']
# sns.pairplot(df_train[cols], size = 2.5)
# plt.show();

In [ ]:
# numerical to categorical
df_train = df_train.replace(
    {
        "MSSubClass" : {20 : "SC20", 30 : "SC30", 40 : "SC40", 45 : "SC45", 
                        50 : "SC50", 60 : "SC60", 70 : "SC70", 75 : "SC75", 
                        80 : "SC80", 85 : "SC85", 90 : "SC90", 120 : "SC120", 
                        150 : "SC150", 160 : "SC160", 180 : "SC180", 190 : "SC190"},
        "MoSold" :     {1 : "Jan", 2 : "Feb", 3 : "Mar", 4 : "Apr", 5 : "May", 6 : "Jun",
                        7 : "Jul", 8 : "Aug", 9 : "Sep", 10 : "Oct", 11 : "Nov", 12 : "Dec"}
    }
)

In [ ]:
# categorical to numerical as it indicates order
df_train = df_train.replace(
    {
        "BsmtCond" : {"Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
        "BsmtExposure" : {"No" : 0, "Mn" : 1, "Av": 2, "Gd" : 3},
        "BsmtFinType1" : {"No" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4, "ALQ" : 5, "GLQ" : 6},
        "BsmtFinType2" : {"No" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4, "ALQ" : 5, "GLQ" : 6},
        "BsmtQual" : {"Po" : 1, "Fa" : 2, "TA": 3, "Gd" : 4, "Ex" : 5},
        "ExterCond" : {"Po" : 1, "Fa" : 2, "TA": 3, "Gd": 4, "Ex" : 5},
        "ExterQual" : {"Po" : 1, "Fa" : 2, "TA": 3, "Gd": 4, "Ex" : 5},
        "FireplaceQu" : {"Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
        "Functional" : {"Sal" : 1, "Sev" : 2, "Maj2" : 3, "Maj1" : 4, "Mod": 5, "Min2" : 6, "Min1" : 7, "Typ" : 8},
        "GarageCond" : { "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
        "GarageQual" : {"Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
        "HeatingQC" : {"Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
        "KitchenQual" : {"Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
        "LandSlope" : {"Sev" : 1, "Mod" : 2, "Gtl" : 3},
        "LotShape" : {"IR3" : 1, "IR2" : 2, "IR1" : 3, "Reg" : 4},
        "PavedDrive" : {"N" : 0, "P" : 1, "Y" : 2},
        "Utilities" : {"ELO" : 1, "NoSeWa" : 2, "NoSewr" : 3, "AllPub" : 4}
    }
)

In [ ]:
# Differentiate numerical features, categorical features and target variable
categorical_features = df_train.select_dtypes(include = ["object"]).columns
numerical_features = df_train.select_dtypes(exclude = ["object"]).columns
numerical_features = numerical_features.drop("SalePrice")
print("Numerical features : " + str(len(numerical_features)))
print("Categorical features : " + str(len(categorical_features)))
train_num = df_train[numerical_features]
train_cat = df_train[categorical_features]
y = df_train.SalePrice

In [ ]:
skewness = train_num.skew()
skewness = skewness[abs(skewness) > 0.5]
print(str(skewness.shape[0]) + " skewed numerical features")
skewed_features = skewness.index
print(skewed_features)

In [ ]:
# 可选，看模型需要
# Log transform of the skewed numerical features to lessen impact of outliers
# Inspired by Alexandru Papiu's script : https://www.kaggle.com/apapiu/house-prices-advanced-regression-techniques/regularized-linear-models
# As a general rule of thumb, a skewness with an absolute value > 0.5 is considered at least moderately skewed
# skewness = train_num.skew()
# skewness = skewness[abs(skewness) > 0.5]
# print(str(skewness.shape[0]) + " skewed numerical features")
# skewed_features = skewness.index
# print(skewed_features)
# train_num[skewed_features] = np.log1p(train_num[skewed_features])

In [ ]:
# 可选，看模型需要
# Create dummy features for categorical values via one-hot encoding
# train_cat = pd.get_dummies(train_cat)

In [ ]:
# output
df_train = pd.concat([train_num, train_cat, y], axis=1)
df_train.to_csv("data/cleaned_train.csv", index = False)